# **Step 1: Load Data and Import Library**

In [ ]:
import pandas as pd
import json
from pandas import DataFrame
!pip install pyvi
from pyvi import ViTokenizer, ViPosTagger
from pyvi import ViUtils
from collections import Counter
import matplotlib.pyplot as plt
import pandas as pd 
import io 
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import f1_score, log_loss
from tabulate import tabulate
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
from nltk.corpus import stopwords

import os
import csv

import warnings
warnings.filterwarnings('ignore')

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')


for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
!apt-get install p7zip
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/train.tsv.7z
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/test.tsv.7z
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/sample_submission.csv.7z

In [ ]:
!unzip /kaggle/input/mercari-price-suggestion-challenge/sample_submission_stg2.csv.zip
!unzip /kaggle/input/mercari-price-suggestion-challenge/test_stg2.tsv.zip

In [ ]:
train_data = pd.read_csv('train.tsv', sep='\t')
test_data = pd.read_csv('test_stg2.tsv', sep='\t')
print(train_data.shape)
print(test_data.shape)
print(train_data.columns)
print(test_data.columns)

In [ ]:
train_data.head()

In [ ]:
test_data.head()

# **Step 2: Data Preprocessing**

**2.1. Check for Duplicates**

In [ ]:
print('Number of duplicates in train: {}'.format(sum(train_data.duplicated())))
print('Number of duplicates in test : {}'.format(sum(test_data.duplicated())))

**2.2. Checking for NaN/null values**

In [ ]:
train_data.isnull().any()

In [ ]:
print('{} NaN/Null values in train'.format(train_data.isnull().values.sum()))
print('{} NaN/Null values in test'.format(test_data.isnull().values.sum()))

In [ ]:
# train_data["category_name"] = train_data["category_name"].fillna("Other").astype("category")
train_data["brand_name"] = train_data["brand_name"].fillna("unknown")
train_data["item_description"] = train_data["item_description"].fillna("None")
train_data["brand_name"] = train_data["brand_name"].astype("category")

# test_data["category_name"] = test_data["category_name"].fillna("Other").astype("category")
test_data["brand_name"] = test_data["brand_name"].fillna("unknown")
test_data["item_description"] = test_data["item_description"].fillna("None")
test_data["brand_name"] = test_data["brand_name"].astype("category")

In [ ]:
train_copy = train_data.copy()
test_copy = test_data.copy()

**2.3. Encoder**

In [ ]:
def split_cat(text):
    try: return text.split("/")
    except: return ("No Label", "No Label", "No Label")
    
def create_split_categories(data):
    data['main_category'],data['subcat_1'],data['subcat_2']=zip(*data['category_name'].apply(lambda x: split_cat(x)))

def toNumeric(frame, data,to):
    le = preprocessing.LabelEncoder()
    frame[to] = le.fit_transform(frame[data].astype(str))
    
def split_cat_encoder(frame):
    toNumeric(frame, 'main_category', 'main_category' )
    toNumeric(frame, 'subcat_1', 'subcat_1' )
    toNumeric(frame, 'subcat_2', 'subcat_2' )
    
def data_frame_encoder(frame):
    toNumeric(frame, 'category_name', 'category_name' )
    toNumeric(frame, 'item_description', 'item_description' )
    toNumeric(frame, 'name', 'name' )
    toNumeric(frame, 'brand_name', 'brand_name' )
    
def divide_cats(data):
    if(0<= data <=1):
        return "CAT1"
    if(1 < data <= 2):
        return "CAT2"
    if(2 < data <= 3):
        return "CAT3"
    if(3 < data <= 4):
        return "CAT4"
    if(4 < data <= 5):
        return "CAT5"
    if(5< data <=6):
        return "CAT6"
    if(6 < data <= 7):
        return "CAT7"
    return "CATOTHER"

def remove_stop_words(x):
    x = ' '.join([i for i in x.lower().split(' ') if i not in stopwords.words('english')])
    return x

In [ ]:
create_split_categories(train_copy)
train_copy['log_price'] = train_copy['price'].map(lambda x: np.log(x) if x>0 else x)
train_copy['price_cats'] = train_copy['log_price'].map(lambda x : divide_cats(x))


test_copy = test_data.copy()
create_split_categories(test_copy)

In [ ]:
train_copy.head(10)

In [ ]:
split_cat_encoder(train_copy)
data_frame_encoder(train_copy)

split_cat_encoder(test_copy)
data_frame_encoder(test_copy)

In [ ]:
train_copy.head(5)

In [ ]:
test_copy.head(5)

# **Step 3 : Exploratory Data Analysis**

In [ ]:
train_copy.price.describe() # std do lech chuan

**Sale Price**

In [ ]:
print(" Range of price : ",'$',train_copy["price"].min(), ' - ','$',train_copy["price"].max())
fig, ax = plt.subplots(2, 1, figsize = (15, 10))
ax[0].hist(train_copy.price, bins = 200, range = [min(train_copy.price), max(train_copy.price)], label = "price",color = "skyblue")
ax[0].set_title("\n \n  Histogram ", fontsize = 15)
ax[0].set_xlabel("Sale Price", fontsize = 10)
ax[0].set_ylabel(" Frequency ", fontsize = 10)
sns.boxplot(train_copy.price, showfliers = False, ax = ax[1],color = "skyblue")
ax[1].set_title("Box Plot", fontsize = 15)
plt.show()

Giá các sản phẩm rơi nhiều vào khoảng 10 đến 30, trong đó lệch về 10 nhiều hơn, có sự xuất hiện của các sản phẩm có giá 0 (do người bán muốn tặng hoặc bán miễn phí).

**Log Price**

In [ ]:
print(" Range of price : ",'$',train_copy["log_price"].min(), ' - ','$',train_copy["log_price"].max())
fig, ax = plt.subplots(2, 1, figsize = (15, 10))
ax[0].hist(train_copy.log_price, bins = 200, range = [min(train_copy.log_price), max(train_copy.log_price)], label = "log_price",color = "skyblue")
ax[0].set_title("\n \n  Histogram ", fontsize = 15)
ax[0].set_xlabel("Log Price", fontsize = 10)
ax[0].set_ylabel(" Frequency ", fontsize = 10)
sns.boxplot(train_copy.log_price, showfliers = False, ax = ax[1],color = "skyblue")
ax[1].set_title("Box Plot", fontsize = 15)
plt.show()

Giá các sản phẩm rơi nhiều vào khoảng 2 đến 4.

Việc lấy log cho price giúp price sẽ được phân bố đồng đều hơn so với việc để nguyên price. Có thể thấy rõ qua Range of price, trong khi Range of price của price là từ $ 0.0  -  $ 2009.0 thì Range of price của log_price chỉ là từ $ 0.0  -  $ 7.60. Việc chênh lệch quá lớn của price (nếu không lấy log) sẽ khiến cho model gặp khó khăn trong việc training, khiến cho model không còn chính xác nữa. Do đó, chúng tôi sẽ chọn log_price thay thế cho price trong quá trình training các model.

**Brand Name**

In [ ]:
# barplot biểu diễn mật độ xuất hiện của các brand name
# scarterplot biểu dễn giá của từng từng sản phảm thuộc từng brand name dưới dạng numberic (index)
brands = train_data["brand_name"].value_counts()
print("Unique Brand Names :", brands.size)
brands_key = []
for i in range(0,5):
    brands_key.append(brands[1:6].keys()[i])
fig, ax = plt.subplots(2, 1, figsize = (15, 10))
sns.barplot(brands[1:6].values, brands_key , ax = ax[0]) #brand[0] có nhãn "-1"
f1 = train_copy['log_price'].values
f2 = train_copy['brand_name'].values

f3 = train_copy['price'].values
f4 = train_copy['brand_name'].values

ax[1].set_title("\n \n  Scatter Plot ", fontsize = 15)
ax[1].scatter(f1, f2, c='black', s = 7)# s là size của scatter
ax[0].set_xlabel(" Appear Times", fontsize = 10)
ax[0].set_ylabel(" Brand Name", fontsize = 10)
ax[1].set_xlabel(" Log Price", fontsize = 10)
ax[1].set_ylabel(" Brand Name", fontsize = 10)

plt.show()

**Item Condition ID**

In [ ]:
fig, ax = plt.subplots(3, 1, figsize = (15,10))
sns.countplot(train_copy.item_condition_id, ax = ax[0])
rectangles = ax[0].patches
ax[0].set_title("Count Plot ", fontsize = 15)
labels = train_copy.item_condition_id.value_counts().values
for rect, label in zip(rectangles, labels):#đưa số lần xuất hiện lên đầu các cột
    height = rect.get_height()
    ax[0].text(rect.get_x() + rect.get_width()/2, height + 5, label, ha = "center", va = "bottom")
    
sns.boxplot(x = train_copy.item_condition_id, y = train_copy.price,showfliers = False, orient = "v", ax = ax[1])
ax[2].scatter(x = train_copy.item_condition_id, y = train_copy.price,alpha=0.9)
ax[2].set_xlabel(" Item Condition id", fontsize = 10)
ax[2].set_ylabel(" Sale Price", fontsize = 10)
plt.show()

**Shipping**

In [ ]:
fig, ax = plt.subplots(3, 1, figsize = (15,10))
sns.countplot(train_copy.shipping, ax = ax[0])
rectangles = ax[0].patches
labels = train_copy.shipping.value_counts().values
for rect, label in zip(rectangles, labels):
    height = rect.get_height()
    ax[0].text(rect.get_x() + rect.get_width()/2, height + 5, label, ha = "center", va = "bottom")
sns.boxplot(x = train_copy.shipping, y = train_copy.price,showfliers = False, orient = "v", ax = ax[1])
ax[2].scatter(x = train_copy.price, y = train_copy.shipping,alpha=0.9)
ax[2].set_xlabel(" Shipping ", fontsize = 10)
ax[2].set_ylabel(" Sale Price", fontsize = 10)
plt.show()

# **Step 4 : Data Modeling**

# **Model 1. Linear Regreession**

In [ ]:
corltn=train_copy.corr() #tính độ tương quan giữa các cột, bỏ qua giá trị null
corltn=corltn.fillna(0)
plt.figure(figsize=(12, 10))
plt.imshow(corltn, cmap='YlGnBu', interpolation='none', aspect='auto')
plt.colorbar()
plt.xticks(range(len(corltn)), corltn.columns, rotation='vertical')
plt.yticks(range(len(corltn)), corltn.columns);
plt.suptitle(' Correlations Heat Map for attributes', fontsize=16, fontweight='bold')

Không có sự tương quan rõ ràng giữa cột giá và các cột khác để sử dụng để dự 
báo.

**Using Category_name**

In [ ]:
X_train, x_test, Y_train, y_test = train_test_split(
train_copy['main_category'], train_copy['log_price'], test_size=0.2, random_state=42) #tách thành 2 tập test và train 
regr_0 = linear_model.LinearRegression()
X_train = X_train[:, np.newaxis] # reshape 2D array into 1D array
Y_train = Y_train[:, np.newaxis]  # reshape 2D array into 1D array
x_test = x_test[:, np.newaxis]  # reshape 2D array into 1D array
y_test = y_test[:, np.newaxis]  # reshape 2D array into 1D array
regr_0.fit(X_train, Y_train)

y_train_predict = regr_0.predict(X_train)
y_val_predict = regr_0.predict(x_test)

table = [['Score', 'Training', 'Valdation'], 
         ['MSE', '{} '.format(round((mean_squared_error(Y_train, y_train_predict)), 5)), '{} '.format(round((mean_squared_error(y_test, y_val_predict)), 5))], 
         ['RMSE', '{} '.format(round((np.sqrt(mean_squared_error(Y_train, y_train_predict))), 2)), '{} '.format(np.sqrt(round((mean_squared_error(y_test, y_val_predict)), 5)))], 
         ['R2_Score', '{} '.format(round((r2_score(Y_train, y_train_predict)), 5)), '{} '.format(round((r2_score(y_test, y_val_predict)), 5))]]

print(tabulate(table, headers='firstrow', tablefmt='grid'))

**Using main category, subcat_1, subcat_2, item_condition_id**

In [ ]:
train_split_data = train_copy[['main_category', 'subcat_1', 'subcat_2', 'item_condition_id']]
X_train, x_test, Y_train, y_test = train_test_split(
train_split_data, train_copy['log_price'], test_size=0.2, random_state=42) #tách thành 2 tập test và train 
regr_1 = linear_model.LinearRegression()
Y_train = Y_train[:, np.newaxis]  # reshape 2D array into 1D array
y_test = y_test[:, np.newaxis]  # reshape 2D array into 1D array
regr_1.fit(X_train, Y_train)
y_predict = regr_1.predict(x_test)

y_train_predict = regr_1.predict(X_train)
y_val_predict = regr_1.predict(x_test)

table = [['Score', 'Training', 'Valdation'], 
         ['MSE', '{} '.format(round((mean_squared_error(Y_train, y_train_predict)), 5)), '{} '.format(round((mean_squared_error(y_test, y_val_predict)), 5))], 
         ['RMSE', '{} '.format(round((np.sqrt(mean_squared_error(Y_train, y_train_predict))), 2)), '{} '.format(np.sqrt(round((mean_squared_error(y_test, y_val_predict)), 5)))], 
         ['R2_Score', '{} '.format(round((r2_score(Y_train, y_train_predict)), 5)), '{} '.format(round((r2_score(y_test, y_val_predict)), 5))]]

print(tabulate(table, headers='firstrow', tablefmt='grid'))

Mean squared error regression loss là rất lớn do đó sử dụng main category, subcat_1, subcat_2, item_condition_id không hiệu quả

**Using main category, subcat_1, subcat_2, item_condition_id, brand_name , shipping, name**

In [ ]:
train_split_data = train_copy[['name','main_category', 'subcat_1', 'subcat_2', 'item_condition_id', 'brand_name', 'shipping']]
X_train, x_test, Y_train, y_test = train_test_split(
train_split_data, train_copy['log_price'], test_size=0.2, random_state=42) #tách thành 2 tập test và train 
regr_2 = linear_model.LinearRegression()
Y_train = Y_train[:, np.newaxis]  # reshape 2D array into 1D array
y_test = y_test[:, np.newaxis]  # reshape 2D array into 1D array
regr_2.fit(X_train, Y_train)

y_train_predict = regr_2.predict(X_train)
y_val_predict = regr_2.predict(x_test)

table = [['Score', 'Training', 'Valdation'], 
         ['MSE', '{} '.format(round((mean_squared_error(Y_train, y_train_predict)), 5)), '{} '.format(round((mean_squared_error(y_test, y_val_predict)), 5))], 
         ['RMSE', '{} '.format(round((np.sqrt(mean_squared_error(Y_train, y_train_predict))), 2)), '{} '.format(np.sqrt(round((mean_squared_error(y_test, y_val_predict)), 5)))], 
         ['R2_Score', '{} '.format(round((r2_score(Y_train, y_train_predict)), 5)), '{} '.format(round((r2_score(y_test, y_val_predict)), 5))]]

print(tabulate(table, headers='firstrow', tablefmt='grid'))

**Using most classes except price, price_cats, log_price**

In [ ]:
train_split_data = train_copy.drop(['price', 'log_price', 'price_cats'], axis = 1)
X_train, x_test, Y_train, y_test = train_test_split(
train_split_data, train_copy['log_price'], test_size=0.2, random_state=42) #tách thành 2 tập test và train 
regr_3 = linear_model.LinearRegression()
Y_train = Y_train[:, np.newaxis]  # reshape 2D array into 1D array
y_test = y_test[:, np.newaxis]  # reshape 2D array into 1D array
regr_3.fit(X_train, Y_train)

y_train_predict = regr_3.predict(X_train)
y_val_predict = regr_3.predict(x_test)

table = [['Score', 'Training', 'Valdation'], 
         ['MSE', '{} '.format(round((mean_squared_error(Y_train, y_train_predict)), 5)), '{} '.format(round((mean_squared_error(y_test, y_val_predict)), 5))], 
         ['RMSE', '{} '.format(round((np.sqrt(mean_squared_error(Y_train, y_train_predict))), 2)), '{} '.format(np.sqrt(round((mean_squared_error(y_test, y_val_predict)), 5)))], 
         ['R2_Score', '{} '.format(round((r2_score(Y_train, y_train_predict)), 5)), '{} '.format(round((r2_score(y_test, y_val_predict)), 5))]]

print(tabulate(table, headers='firstrow', tablefmt='grid'))

Việc sử dụng tất cả các class (trừ mấy class về price) đã giảm được MSE, RMSE đi so với việc sử dụng một vài class.

Mặc dù cho ra kết quả MSE khá là thấp nhưng tôi vẫn muốn giảm MSE (RMSE) đi nữa. Do đó, tôi thay vì sử dụng LabelEncoder để Label Encoder cho tập dữ liệu thì tôi sử dụng CountVectorizer cho dữ liệu chữ, LabelBinarizer cho dữ liệu số, TfidfVectorizer cho riêng cột item_description, sau đó sử dụng hstack để gộp lại dữ liệu sau khi convert ở trên sang dạng dữ liệu Compressed Sparse Row matrix để tạo ra tập dữ liệu training.

In [ ]:
train_csr_matrix = train_data.copy()
create_split_categories(train_csr_matrix)

test_csr_matrix = test_data.copy()
create_split_categories(test_csr_matrix)

**CountVectorizer and LabelBinarizer**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelBinarizer
from scipy.sparse import csr_matrix

vectorizer = CountVectorizer()
train_name = vectorizer.fit_transform(train_csr_matrix['name'])
test_name = vectorizer.transform(test_csr_matrix['name'])

lb_condition = LabelBinarizer(sparse_output=True)
train_condition = lb_condition.fit_transform(train_csr_matrix['item_condition_id'])
test_condition = lb_condition.transform(test_csr_matrix['item_condition_id'])

vectorizer_brand = CountVectorizer()
train_brand = vectorizer_brand.fit_transform(train_csr_matrix['brand_name'])
test_brand = vectorizer_brand.transform(test_csr_matrix['brand_name'])

lb_shipping = LabelBinarizer(sparse_output=True)
train_shipping = lb_condition.fit_transform(train_csr_matrix['shipping'])
test_shipping = lb_condition.transform(test_csr_matrix['shipping'])

vectorizer_main_category = CountVectorizer()
train_main_category = vectorizer_main_category.fit_transform(train_csr_matrix['main_category'])
test_main_category = vectorizer_main_category.transform(test_csr_matrix['main_category'])

vectorizer_subcat_1 = CountVectorizer()
train_subcat_1 = vectorizer_subcat_1.fit_transform(train_csr_matrix['subcat_1'])
test_subcat_1 = vectorizer_subcat_1.transform(test_csr_matrix['subcat_1'])

vectorizer_subcat_2 = CountVectorizer()
train_subcat_2 = vectorizer_subcat_2.fit_transform(train_csr_matrix['subcat_2'])
test_subcat_2 = vectorizer_subcat_2.transform(test_csr_matrix['subcat_2'])


In [ ]:
# train_csr_matrix['item_description'] = train_csr_matrix['item_description'].apply(remove_stop_words)
# test_csr_matrix['item_description'] = test_csr_matrix['item_description'].apply(remove_stop_words)

In [ ]:
# tfidf_descp = TfidfVectorizer(ngram_range=(1,2),min_df=10,max_features=5000)
# train_descp = tfidf_descp.fit_transform(train_csr_matrix['item_description'])
# test_descp = tfidf_descp.transform(test_csr_matrix['item_description'])

In [ ]:
train_csr_matrix['log_price'] = train_csr_matrix['price'].map(lambda x: np.log(x) if x>0 else x)

In [ ]:
train_matrix_list = (train_name, train_brand, train_condition,
                      train_shipping, train_main_category, 
                      train_subcat_1, train_subcat_2)

test_matrix_list = (test_name, test_brand, test_condition,
                      test_shipping, test_main_category, 
                      test_subcat_1, test_subcat_2)

In [ ]:
from scipy.sparse import hstack
data_train_matrix = hstack(train_matrix_list).tocsr()

In [ ]:
#Training model
X_train, x_test, Y_train, y_test = train_test_split(
data_train_matrix, train_csr_matrix['log_price'], test_size=0.2, random_state=42) #tách thành 2 tập test và train 
regr_csr_matrix = linear_model.LinearRegression(normalize=True)

regr_csr_matrix.fit(X_train, Y_train)

y_train_predict = regr_csr_matrix.predict(X_train)
y_val_predict = regr_csr_matrix.predict(x_test)

In [ ]:
table = [['Score', 'Training', 'Valdation'], 
         ['MSE', '{} '.format(round((mean_squared_error(Y_train, y_train_predict)), 5)), '{} '.format(round((mean_squared_error(y_test, y_val_predict)), 5))], 
         ['RMSE', '{} '.format(round((np.sqrt(mean_squared_error(Y_train, y_train_predict))), 2)), '{} '.format(np.sqrt(round((mean_squared_error(y_test, y_val_predict)), 5)))], 
         ['R2_Score', '{} '.format(round((r2_score(Y_train, y_train_predict)), 5)), '{} '.format(round((r2_score(y_test, y_val_predict)), 5))]]

print(tabulate(table, headers='firstrow', tablefmt='grid'))

MSE, RMSE đã giảm nhiều; R2_Score cũng tăng so với việc sử dụng LabelEncoder. Tuy vậy, việc sử dụng Compressed Sparse Row khiến cho model chạy lâu hơn.

# **Model 2: Random Forest Regression**

In [ ]:
X_train_rfr = train_copy.drop(['price', 'log_price' , 'price_cats'], axis = 1)
Y_train_rfr = train_copy['log_price']

In [ ]:
# #Train model
X_train, x_test, Y_train, y_test = train_test_split(
X_train_rfr, Y_train_rfr, test_size=0.2, random_state=42) #tách thành 2 tập test và train 

rfr = RandomForestRegressor(n_jobs = -1, min_samples_leaf = 5, n_estimators = 200)
rfr.fit(X_train, Y_train)

In [ ]:
y_train_predict = rfr.predict(X_train)
y_val_predict = rfr.predict(x_test)

table = [['Score', 'Training', 'Valdation'], 
         ['MSE', '{} '.format(round((mean_squared_error(Y_train, y_train_predict)), 5)), '{} '.format(round((mean_squared_error(y_test, y_val_predict)), 5))], 
         ['RMSE', '{} '.format(round((np.sqrt(mean_squared_error(Y_train, y_train_predict))), 2)), '{} '.format(np.sqrt(round((mean_squared_error(y_test, y_val_predict)), 5)))], 
         ['R2_Score', '{} '.format(round((r2_score(Y_train, y_train_predict)), 5)), '{} '.format(round((r2_score(y_test, y_val_predict)), 5))]]

print(tabulate(table, headers='firstrow', tablefmt='grid'))

Hai model trên đã cho ra được kết quả MSE (RMSE) khá tốt (đặc biệt là sử dụng việc CountVectorizer, LabelBinarizer, TfidfVectorizer)chia log_price ra thành 8 khoảng giá tiền.

# **Model 3: K-MEANS**

In [ ]:
price_cats = train_copy.price_cats
def Kmeansmethod(frame1,frame1_name):
    np.random.seed(5)
    X = frame1
    y = price_cats
    estimators = [('k_means_iris_11', KMeans(n_clusters=11)),
              ('k_means_iris_8', KMeans(n_clusters=8)),
              ('k_means_iris_3', KMeans(n_clusters=3)),
              ('k_means_iris_bad_init', KMeans(n_clusters=3, n_init=1,
                                               init='random'))]
    X  = X.values
    y = y.values
    colors = {'CAT1':'#1f77b4', 'CAT2':'#ff7f0e', 'CAT3':'#2ca02c', 'CAT4':'#d62728', 'CAT5':'#9467bd', 'CAT6' : '#8c564b', 'CAT7':'#e377c2', 'CATOTHER' : "#1e12c4", 'CAT50':'#17becf'}
    cat_color = price_cats.apply(lambda x : colors[x])
    cat_color = cat_color.to_list()
    fignum = 1
    titles = ['11 clusters' , '8 clusters', '3 clusters', '3 clusters, bad initialization']
    for name, est in estimators:
        fig = plt.figure(fignum, figsize=(4, 3))
        ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
        est.fit(X)
        labels = est.labels_

        ax.scatter(X[:, 3], X[:, 0], X[:, 2],
                   c=labels.astype(np.float), edgecolor='k')

        ax.w_xaxis.set_ticklabels([])
        ax.w_yaxis.set_ticklabels([])
        ax.w_zaxis.set_ticklabels([])
        ax.set_xlabel(frame1_name[3])
        ax.set_ylabel(frame1_name[0])
        ax.set_zlabel(frame1_name[2])
        ax.set_title(titles[fignum - 1])
        ax.dist = 12
        fignum = fignum + 1


    fig = plt.figure(fignum, figsize=(4, 3))
    ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)

    # Labels reorder

    ax.scatter(X[:, 3], X[:, 0], X[:, 2], c=cat_color, edgecolor='k')

    ax.w_xaxis.set_ticklabels([])
    ax.w_yaxis.set_ticklabels([])
    ax.w_zaxis.set_ticklabels([])
    ax.set_xlabel(frame1_name[3])
    ax.set_ylabel(frame1_name[0])
    ax.set_zlabel(frame1_name[2])
    ax.set_title('Sale Price Category')
    ax.dist = 12

    fig.show()

In [ ]:
frame1=train_copy[['main_category','subcat_1','subcat_2','brand_name']]
frame1_name = ['main_category','subcat_1','subcat_2','brand_name']
Kmeansmethod(frame1,frame1_name)

Nhận xét: Ba hình (Hình 1, 2, 3) sử dụng model k-mean ( cluster lần lượt bằng 11, 8, 3). Ta thấy, dữ liệu được chia thành các cụm tương ứng với số cluster. Trong khi đó dữ liệu thực tế (Hình 4), các điểm dữ liệu có màu sắc tương ứng với label là khoảng giá tiền nằm ngẫu nhiên không theo một quy luật hay một cụm nào cả. Do đó, không thể lấy tính tương đồng của các điểm dữ liệu ở trong cùng một cụm để gán nhãn cụm theo nhãn (label) là khoảng giá tiền.

# **Model 4: K-NN**

In [ ]:
train_encoder_price_category = train_copy.copy()
toNumeric(train_encoder_price_category, 'price_cats', 'price_cats')
train_encoder_price_category.head(5)

In [ ]:
price_cats = np.array(train_encoder_price_category.pop('price_cats'))#tách cột price_cats từ frametrain
train_encoder_price_category = train_encoder_price_category.drop(['train_id', 'price', 'category_name'] , axis=1)
train_encoder_price_category.head(5)

In [ ]:
def Knnmethod(frametrain1,price_cats):
    frametraink1 = frametrain1.to_numpy()
    X_train, X_test, y_train, y_test = train_test_split(frametraink1, price_cats, test_size=0.20, random_state=42)
    accuracy_array = []
    k_array = []
    for k in range(1,100,3):
        knn = KNeighborsClassifier(n_neighbors=k, p = 2)
        accuracy = cross_val_score(knn, X_train, y_train, cv=10, scoring='accuracy')#cv = 10 fold, mặc định là 5
        accuracy_array.append(accuracy.mean())#do chia làm nhiều fold nên accurracy là một arr cần phải lấy mean
        k_array.append(k)
        
    class_error = 1.0 - np.array(accuracy_array)
    plt.plot(k_array, class_error)
    plt.xlabel('K')
    plt.ylabel('Classification Error')
    plt.show()
    min_ind = np.argmin(class_error)
    OptK = k_array[min_ind]

    accuracy_array = []
    k_array = []
    for k in range(OptK-2, OptK+2,1):
        knn = KNeighborsClassifier(n_neighbors=k)
        accuracy = cross_val_score(knn, X_train, y_train, cv=10, scoring='accuracy')#cv = 10 fold, mặc định là 5
        accuracy_array.append(accuracy.mean())#do chia làm nhiều fold nên accurracy là một arr cần phải lấy mean
        k_array.append(k)
    class_error = 1.0 - np.array(accuracy_array)
    plt.plot(k_array, class_error)
    plt.xlabel('K')
    plt.ylabel('Classification Error')
    plt.show()
    min_ind = np.argmin(class_error)
    OptK = k_array[min_ind]

    print ("Optimal value of K is %d " %  OptK)
    knn = KNeighborsClassifier(n_neighbors=OptK)

    # fitting the model
    knn.fit(X_train, y_train)

    # predict
    pred = knn.predict(X_test)

    # evaluate accuracy
    print("accuracy_score",accuracy_score(y_test, pred))

In [ ]:
Knnmethod(train_encoder_price_category[0:5000],price_cats[0:5000])

In [ ]:
frametrain=train_encoder_price_category[['brand_name','main_category']]
Knnmethod(frametrain[0:5000],price_cats[0:5000])

In [ ]:
frametrain=train_encoder_price_category[['main_category','subcat_1','brand_name']]
Knnmethod(frametrain[0:5000],price_cats[0:5000])

In [ ]:
frametrain=train_encoder_price_category[['main_category','subcat_1','subcat_2', 'item_condition_id']]
Knnmethod(frametrain[0:5000],price_cats[0:5000])

In [ ]:
frametrain=train_encoder_price_category[['main_category','subcat_1','subcat_2', 'item_condition_id', 'brand_name']]
Knnmethod(frametrain[0:5000],price_cats[0:5000])

In [ ]:
frametrain=train_encoder_price_category[['main_category','subcat_1','subcat_2', 'item_condition_id', 'brand_name','shipping']]
Knnmethod(frametrain[0:5000],price_cats[0:5000])

Nhận xét: Khi dùng K-NN model cho tập dữ liệu, chúng tôi loại bỏ cột 'Price' và sử dụng cột 'price_cats' làm labels kết quả cho mô hình, do 2 cột này thể hiện độ tương quan cao. Sau khi áp dụng K-NN model cho tập dữ liệu, chúng tôi nhận được độ chính xác thấp, trong khoảng 0.18-0.225, giá trị chính xác cao nhất sau khi sử dụng các cột 'main_category', 'subcat_1', 'brand_name'.

# **Model 5: LogisticRegression**

In [ ]:
train_encoder_price_category.head(5)

In [ ]:
X_train_logr = train_encoder_price_category.drop(['log_price'], axis = 1)
Y_train_logr = price_cats

In [ ]:
#Training
X_train, x_test, Y_train, y_test = train_test_split(
X_train_logr, Y_train_logr, test_size=0.2, random_state=42) #tách thành 2 tập test và train 

Y_train = Y_train[:, np.newaxis]  # reshape 2D array into 1D array
y_test = y_test[:, np.newaxis]  # reshape 2D array into 1D array

lr = LogisticRegression(random_state=0).fit(X_train, Y_train)


In [ ]:
y_train_predict = lr.predict(X_train)
y_val_predict = lr.predict(x_test)

table = [['Score', 'Training', 'Valdation'], ['Accuracy', '{} %'.format(round((accuracy_score(Y_train, y_train_predict)*100), 2)), '{} %'.format(round((accuracy_score(y_test, y_val_predict)*100), 2))], ['F1_Score', '{} %'.format(round((f1_score(Y_train, y_train_predict, average='macro')*100))), '{} %'.format(round((f1_score(y_test, y_val_predict, average='macro')*100)))]]
print(tabulate(table, headers='firstrow', tablefmt='grid'))

**Có thể thấy Accuracy Score và F1_Score khi sử dụng model LogisticRegression với các class được chia theo từng khoảng giá tiền là rất thấp => Việc sử dụng model LogisticRegression theo class từng khoảng giá tiền là không hiệu quả.**

# **Step 5: Test & Output**

In [ ]:
data_test_matrix = hstack(test_matrix_list).tocsr()
preds = regr_csr_matrix.predict(data_test_matrix)
np.exp(preds)
submission_data = pd.read_csv('sample_submission_stg2.csv')
submission_data.loc[:, 'price'] = np.expm1(preds)
submission_data.to_csv('submission.csv', index=False)